In [141]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
import os
import sys
import json
import glob
import random
from random import shuffle
import time
import re
import math
import collections
from tqdm import tqdm
from config import EXPERIMENT, RUN_NAME, ARTIFACT_DIR
from logger import logger
import mlflow.tensorflow

import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

import dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from sklearn import model_selection as sk_model_selection
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [143]:
EPOCH = 40

In [144]:
run_params = {'experiment': EXPERIMENT,
              'iteration': RUN_NAME,
              'epoch': EPOCH,
              'artifact_dir': ARTIFACT_DIR}

In [184]:
UNZIP_DATA = False
PROCESS_DATA = True
LOAD_DATA = False
data_dir = "./Data"
mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

In [185]:
%%capture
if (UNZIP_DATA):
    !unzip "Data/rsna-miccai-brain-tumor-radiogenomic-classification.zip" -d "Data"

In [186]:
if (PROCESS_DATA):
    train_df = pd.read_csv(os.path.join(data_dir, "train_labels.csv" ))
    train_df = train_df[~train_df.BraTS21ID.isin(excluded_images)]
    test_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
sample_submission = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

In [187]:
#scalling the image to {0,1}
def load_image(path, size = 224):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

In [188]:
def get_all_image_paths(brats21id, image_type, folder='train'): 
    if (image_type in mri_types):
        patient_path = os.path.join(data_dir, folder, str(brats21id).zfill(5))
        paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")),key=lambda x: int(x[:-4].split("-")[-1]))
        return np.array(paths[0:len(paths):1])
    return None

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_image(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]

In [189]:
def get_all_data_for_train(train_df, image_type, image_size=32):    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', image_size)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

In [190]:
def get_all_data_for_test(test_df, image_type, image_size=32):
    X = []
    test_ids = []
    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', image_size)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

In [191]:
if (PROCESS_DATA):
    X, y, trainidt = get_all_data_for_train(train_df, 'T1wCE')
    X_test, testidt = get_all_data_for_test(test_df, 'T1wCE')
    X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = sk_model_selection.train_test_split(X, y, trainidt, test_size=0.2, random_state=42)

100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [01:10<00:00,  1.23it/s]


In [192]:
if (PROCESS_DATA):
    np.save("x_train_data.npy", X_train)
    np.save("x_valid_data.npy", X_valid)
    np.save("y_train_data.npy", y_train)
    np.save("y_valid_data.npy",  y_valid)
elif (LOAD_DATA): 
    X_train = np.load("x_train_data.npy")
    X_valid = np.load("x_valid_data.npy")
    y_train = np.load("y_train_data.npy")
    y_valid = np.load("y_valid_data.npy")

In [193]:
#https://www.tensorflow.org/api_docs/python/tf/expand_dims
X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

In [194]:
#https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

print(f"X_train data: Rows={X_train.shape[0]}, Columns={X_train.shape[1]}, Type={type(X_train)}")
print(f"X_valid data: Rows={X_valid.shape[0]}, Columns={X_valid.shape[1]}, Type={type(X_valid)}")
print(f"y_train data: Rows={y_train.shape[0]}, Columns={y_train.shape[1]}, Type={type(y_train)}")
print(f"y_valid data: Rows={y_valid.shape[0]}, Columns={y_valid.shape[1]}, Type={type(y_valid)}")

X_train data: Rows=41019, Columns=32, Type=<class 'tensorflow.python.framework.ops.EagerTensor'>
X_valid data: Rows=10255, Columns=32, Type=<class 'tensorflow.python.framework.ops.EagerTensor'>
y_train data: Rows=41019, Columns=2, Type=<class 'numpy.ndarray'>
y_valid data: Rows=10255, Columns=2, Type=<class 'numpy.ndarray'>


## Models

In [195]:
def get_model1(name='model1'):
    np.random.seed(0)
    random.seed(42)
    tf.random.set_seed(42)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)

    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output, name=name)
    #https://www.tensorflow.org/api_docs/python/tf/keras/metrics/AUC
    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc])
    return model

In [202]:
# source: https://keras.io/examples/vision/3D_image_classification/
def get_model2(width=128, height=128, depth=64, name='model2'):
    
    inputs_model = tf.keras.Input((width, height, depth, 1))
    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs_model)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(units=512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    outputs_model = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs_model, outputs_model, name=name)
    
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
    return model

In [209]:
def get_model3(name='model3'):
    np.random.seed(0)
    random.seed(42)
    tf.random.set_seed(42)

    input_model = keras.Input(shape=X_train.shape[1:])
    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(input_model)
    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)
    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)
    h = keras.layers.Dropout(0.1)(h)
    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)
    output_model = keras.layers.Dense(2, activation="softmax")(h)
    model = keras.Model(input_model, output_model, name=name)

    # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay
     
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
         initial_learning_rate=0.0001,
         decay_steps=100000,
         decay_rate=0.96, 
         staircase=True
     )
  
    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
    model.compile(loss="categorical_crossentropy", 
                  optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), 
                  metrics=[roc_auc])
    return model

In [210]:
checkpoint_filepath = "best_model.h5"
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="val_roc_auc",
    mode="max",
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
)

In [211]:
## Early stopping 
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_roc_auc", mode='max', patience=3)

In [212]:
model1 = get_model1()
model1.summary()

Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
rescaling_8 (Rescaling)      (None, 32, 32, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 29, 29, 64)        1088      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 13, 13, 32)        8224      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 13, 13, 32)        0    

In [213]:
model2 = get_model2()
model2.summary()

Model: "model2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 128, 128, 64, 1)] 0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 126, 126, 62, 64)  1792      
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 63, 63, 31, 64)    0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 63, 63, 31, 64)    256       
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 61, 61, 29, 64)    110656    
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 30, 30, 14, 64)    0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 30, 14, 64)    256  

In [214]:
model3 = get_model3() 
model3.summary()

Model: "model3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
rescaling_9 (Rescaling)      (None, 32, 32, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 29, 29, 64)        1088      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 13, 13, 32)        8224      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 13, 13, 32)        0    

In [215]:
history = model.fit(x=X_train, y = y_train, epochs=EPOCH, 
                    callbacks=[model_checkpoint_cb, early_stopping_cb],
                    validation_data=(X_valid, y_valid))

Epoch 1/40
1282/1282 [==============================] - 40s 31ms/step - loss: 0.4978 - roc_auc: 0.8282 - val_loss: 0.4304 - val_roc_auc: 0.8640

Epoch 00001: val_roc_auc improved from -inf to 0.86398, saving model to best_model.h5
Epoch 2/40
1282/1282 [==============================] - 40s 31ms/step - loss: 0.4080 - roc_auc: 0.8752 - val_loss: 0.4212 - val_roc_auc: 0.8737

Epoch 00002: val_roc_auc improved from 0.86398 to 0.87370, saving model to best_model.h5
Epoch 3/40
1282/1282 [==============================] - 40s 31ms/step - loss: 0.3710 - roc_auc: 0.8963 - val_loss: 0.3884 - val_roc_auc: 0.8903

Epoch 00003: val_roc_auc improved from 0.87370 to 0.89025, saving model to best_model.h5
Epoch 4/40
1282/1282 [==============================] - 39s 30ms/step - loss: 0.3447 - roc_auc: 0.9109 - val_loss: 0.3865 - val_roc_auc: 0.8953

Epoch 00004: val_roc_auc improved from 0.89025 to 0.89530, saving model to best_model.h5
Epoch 5/40
1282/1282 [==============================] - 41s 32ms/st

In [216]:
#https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model
model_best = tf.keras.models.load_model(filepath=checkpoint_filepath)

In [218]:
y_pred = model_best.predict(X_valid)
pred = np.argmax(y_pred, axis=1)
result = pd.DataFrame(trainidt_valid)
result[1] = pred

result.columns = ["BraTS21ID", "MGMT_value"]
result2 = result.groupby("BraTS21ID", as_index=False).mean()

result2 = result2.merge(train_df, on="BraTS21ID")
auc = roc_auc_score(
    result2.MGMT_value_y,
    result2.MGMT_value_x,
)
print(f"Validation AUC={auc}")
#log_mlflow(run_params, model_best, y_valid, pred)

Validation AUC=0.9818550817827434


In [222]:
y_pred = model_best.predict(X_test)

pred = np.argmax(y_pred, axis=1) #

result = pd.DataFrame(testidt)
result[1] = pred
np.set_printoptions(threshold=sys.maxsize)
print(pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0
 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1
 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0
 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1
 1 1 0 0 0 0 0 0 0 0 0 0 

## Submission 

In [223]:
result.columns=['BraTS21ID','MGMT_value']

result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample_submission['BraTS21ID']

# Rounding... 0.907866 -> 0.9
result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'] = result2['MGMT_value'] # No rounding
result2.to_csv('submission_vd.csv',index=False)
result2

,BraTS21ID,MGMT_value
0,1,0.4
1,13,0.2
2,15,0.5
3,27,0.4
4,37,0.6
...,...,...
82,826,0.5
83,829,0.7
84,833,0.4
85,997,0.2
